<a href="https://colab.research.google.com/github/Siu0901/AI_study/blob/main/LeNet_5_%EA%B5%AC%ED%98%84.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import torch
from torch import nn
import torch.optim as optim
from torch.utils.data import DataLoader
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt

In [6]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

cpu


In [7]:
# 하이퍼 파라미터 설정
batch_size = 64
learning_rate = 0.001
num_epochs = 5

In [8]:
# LeNet-5는 32*32로 받으니까 MNIST 데이터 크기 늘려줌
transforms = transforms.Compose([
    transforms.Resize((32,32)),
    transforms.ToTensor(), # 0 ~ 255를 0~1로 만들어줌
    transforms.Normalize((0.1307,), (0.3081,)) # -1~ 1로 만들어줌
    # 이렇게 정규화 해줘야 데이터가 0 중심으로 모여 학습 속도 빠름
])

In [11]:
# 데이터셋 다운로드
train_data = torchvision.datasets.MNIST(root='./data', train=True, download=True, transform=transforms)
test_data = torchvision.datasets.MNIST(root='./data', train=False, download=True, transform=transforms)

In [12]:
# 데이터 로드
train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=False)

In [13]:
# LeNet-5 모델 정의
class LeNet_5(nn.Module):
  def __init__(self):
    super(LeNet_5, self).__init__()
    self.Con = nn.Sequential(
        nn.Conv2d(1, 6, 5),
        nn.ReLU(),
        nn.MaxPool2d(2,2),
        nn.Conv2d(6,16,5),
        nn.ReLU(),
        nn.MaxPool2d(2,2),
        nn.Conv2d(16, 120, 5),
        nn.ReLU()
    )
    self.Full = nn.Sequential(
        nn.Flatten(),
        nn.Linear(120, 84),
        nn.ReLU(),
        nn.Linear(84,10),
    )
  def forward(self, x):
    x = self.Con(x)
    x = self.Full(x)
    return x

In [14]:
model = LeNet_5().to(device)

In [15]:
# 손실 함수 및 옵티마이저 설정
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [17]:
# 학습 루프문
for epo in range(num_epochs):
  model.train() # 학습 모드 전환
  running_loss = 0.0 # 학습 중에 평균 점수 볼려고 하는거임

  for i, (images, labels) in enumerate(train_loader):
    images, labels = images.to(device), labels.to(device)

    optimizer.zero_grad() # 기울기 초기화 해주고

    output = model(images) # 순전파 과정

    loss = criterion(output, labels) # 손실 즉 기울기 계산 하고

    loss.backward()
    optimizer.step()

    running_loss += loss.item()

    if (i+1) % 100 == 0:
      print(f'Epoch [{epo+1}/{num_epochs}], Step [{i+1}/{len(train_loader)}], Loss: {running_loss/100:.4f}')
      running_loss = 0.0


Epoch [1/5], Step [100/938], Loss: 0.9357
Epoch [1/5], Step [200/938], Loss: 0.2630
Epoch [1/5], Step [300/938], Loss: 0.1916
Epoch [1/5], Step [400/938], Loss: 0.1274
Epoch [1/5], Step [500/938], Loss: 0.1125
Epoch [1/5], Step [600/938], Loss: 0.0998
Epoch [1/5], Step [700/938], Loss: 0.0836
Epoch [1/5], Step [800/938], Loss: 0.0828
Epoch [1/5], Step [900/938], Loss: 0.0785
Epoch [2/5], Step [100/938], Loss: 0.0552
Epoch [2/5], Step [200/938], Loss: 0.0632
Epoch [2/5], Step [300/938], Loss: 0.0624
Epoch [2/5], Step [400/938], Loss: 0.0555
Epoch [2/5], Step [500/938], Loss: 0.0572
Epoch [2/5], Step [600/938], Loss: 0.0539
Epoch [2/5], Step [700/938], Loss: 0.0527
Epoch [2/5], Step [800/938], Loss: 0.0517
Epoch [2/5], Step [900/938], Loss: 0.0521
Epoch [3/5], Step [100/938], Loss: 0.0464
Epoch [3/5], Step [200/938], Loss: 0.0413
Epoch [3/5], Step [300/938], Loss: 0.0423
Epoch [3/5], Step [400/938], Loss: 0.0386
Epoch [3/5], Step [500/938], Loss: 0.0380
Epoch [3/5], Step [600/938], Loss:

In [18]:
# 테스트 및 정확도 평가

model.eval() # 평가모드로 전환
with torch.no_grad(): # 기울기 계산 비활성화 하는거
  correct = 0
  total = 0
  for images, labels in test_loader:
    images, labels = images.to(device), labels.to(device)
    outputs = model(images)

    _, predicted = torch.max(outputs.data, 1)

    total += labels.size(0)
    correct += (predicted == labels).sum().item()
  print(f"테스트 데이터셋 정확도: {100 * correct / total}%")

테스트 데이터셋 정확도: 98.93%
